In [6]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [16]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

## cluster composition and centroids
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)

result = dict(zip(list(corr_matrix.columns), process.apply_SPONGE(correlation_matrix=corr_matrix, k=number_of_clusters))) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()
## constituent_weights

sigma = 0.01

In [19]:
int(df_cleaned['AA']['Cluster'])

26

In [20]:
for ticker in df_cleaned.columns:
    df_cleaned[ticker][:-1] = df_cleaned[ticker][:-1] - centroid_returns[int(df_cleaned[ticker]['Cluster'])]

In [80]:
# Mergez df_cleaned avec centroid_returns_reset sur la colonne 'Cluster'
merged_df = pd.merge(df_cleaned, centroid_returns_reset, on='Cluster')

merged_df

,03/01/2000_x,04/01/2000_x,05/01/2000_x,06/01/2000_x,07/01/2000_x,10/01/2000_x,11/01/2000_x,12/01/2000_x,13/01/2000_x,14/01/2000_x,...,17/12/2020_y,18/12/2020_y,21/12/2020_y,22/12/2020_y,23/12/2020_y,24/12/2020_y,28/12/2020_y,29/12/2020_y,30/12/2020_y,31/12/2020_y
0,-0.013042,0.010043,0.047628,-0.011713,-0.016118,-0.032073,0.022608,-0.005249,-0.018210,-0.020109,...,0.003021,-0.002933,0.011263,-0.004149,0.002343,0.000279,-0.007297,-0.011804,0.007499,0.003173
1,-0.070381,0.013986,0.017212,0.040410,-0.013334,-0.006711,-0.006734,0.006734,0.013334,-0.026668,...,0.003021,-0.002933,0.011263,-0.004149,0.002343,0.000279,-0.007297,-0.011804,0.007499,0.003173
2,-0.015123,-0.021381,0.005877,0.048790,0.065452,-0.021661,-0.039368,0.007547,-0.001892,-0.019121,...,0.003021,-0.002933,0.011263,-0.004149,0.002343,0.000279,-0.007297,-0.011804,0.007499,0.003173
3,-0.056281,0.000000,0.068468,0.019371,-0.002395,-0.012180,-0.014528,0.007290,0.035932,0.002339,...,0.003021,-0.002933,0.011263,-0.004149,0.002343,0.000279,-0.007297,-0.011804,0.007499,0.003173
4,-0.012185,0.026069,0.036750,0.044980,0.003314,0.028446,-0.002414,-0.005675,-0.008163,0.000000,...,0.003021,-0.002933,0.011263,-0.004149,0.002343,0.000279,-0.007297,-0.011804,0.007499,0.003173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,-0.036076,-0.024898,0.019526,0.001388,0.043172,0.037118,0.051565,0.034246,0.029270,0.053069,...,0.005441,-0.007127,0.004916,0.002575,0.001781,0.003469,0.000425,-0.006979,-0.001052,0.004569
691,-0.032576,-0.025106,0.013514,-0.001679,0.008382,0.001674,-0.006734,0.003361,0.006689,0.016584,...,0.005441,-0.007127,0.004916,0.002575,0.001781,0.003469,0.000425,-0.006979,-0.001052,0.004569
692,0.007968,0.000000,0.000000,0.000000,0.015625,-0.015748,-0.007905,0.008032,0.008032,-0.008032,...,0.005441,-0.007127,0.004916,0.002575,0.001781,0.003469,0.000425,-0.006979,-0.001052,0.004569
693,-0.025613,-0.041118,-0.012453,0.034233,0.014458,0.019048,0.017522,-0.025376,0.027399,0.006674,...,0.005441,-0.007127,0.004916,0.002575,0.001781,0.003469,0.000425,-0.006979,-0.001052,0.004569


In [81]:
# Sélectionnez uniquement les colonnes de rendements de tickers et de centroid_returns
tickers_returns = merged_df.iloc[:, :len(df_cleaned.columns)]


In [82]:
centroid_returns_cluster = merged_df.iloc[:, len(df_cleaned.columns):]

# Soustrayez le rendement moyen du cluster au rendement de chaque ticker
difference_returns = tickers_returns.sub(centroid_returns_cluster.mean(axis=1), axis=0).drop('Cluster', axis=1)
difference_returns.columns = difference_returns.columns.str.replace('_x', '')

In [92]:
norm_values = np.exp(-np.square(difference_returns).sum(axis=1) * 1e-6 / (sigma**2))
np.set_printoptions(precision=4, suppress=True)
# Créez un nouveau DataFrame avec la colonne de normes
compressed_df = pd.DataFrame({'Norm': norm_values})

In [93]:
compressed_df

,Norm
0,0.756743
1,0.762957
2,0.769503
3,0.769592
4,0.768710
...,...
690,0.902000
691,0.903796
692,0.911668
693,0.889666
